In [1]:
import requests,json,re,datetime
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

## Please enter your linkedin account username and password in the login_information dictionary that is created in the next cell

In [2]:
client = requests.Session()


HOMEPAGE_URL = 'https://www.linkedin.com'
LOGIN_URL = 'https://www.linkedin.com/uas/login-submit'
CONNECTIONS_URL = 'https://www.linkedin.com/mynetwork/invite-connect/connections/'
ASPIRING_DATA_SCIENTIST = 'https://www.linkedin.com/search/results/people/?facetGeoUrn=%5B%22102713980%22%5D&keywords=data%20scientist&origin=FACETED_SEARCH'
                            #'https://www.linkedin.com/search/results/people/?keywords=Data%20Scientist&origin=SWITCH_SEARCH_VERTICAL'#get url, soup object and csrf token value
html = client.get(HOMEPAGE_URL).content
soup = BeautifulSoup(html, "html.parser")
csrf = soup.find('input', dict(name='loginCsrfParam'))['value']

#create login parameters
login_information = {
    'session_key':'username',
    'session_password':'password',
    'loginCsrfParam': csrf,
}
#try and login
try:
    client.post(LOGIN_URL, data=login_information)
    print("Login Successful")
except:
    print("Failed to Login")

Login Successful


In [3]:
def conns(keyword):
    parts=keyword.split(' ')
    middle='%20'.join(parts)
    first='https://www.linkedin.com/search/results/people/?keywords='+middle+'&origin=SUGGESTION&page='
    #first='https://www.linkedin.com/search/results/people/?keywords='+middle+'&origin=GLOBAL_SEARCH_HEADER&page='
    return first

In [4]:
l=conns('Neural networks')
l

'https://www.linkedin.com/search/results/people/?keywords=Neural%20networks&origin=SUGGESTION&page='

In [4]:
def url_construction(link,pageno):
    url=link+str(pageno)
    html = client.get(url).content #opens ASPIRING_DATA_SCIENTIEST
    print(url)
    soup = BeautifulSoup(html , "html.parser") 
    return soup

In [6]:
soup=url_construction(l,1)

https://www.linkedin.com/search/results/people/?keywords=Neural%20networks&origin=SUGGESTION&page=1


In [5]:
def get_df(k,p):
    names=[]
    exp=[]
    urls=[]
    locs=[]
    d=pd.DataFrame()
    for i in range(10):
        if(p==1):
            x=k['data']['elements'][1]['elements'][i]
        else:
            x=k['data']['elements'][0]['elements'][i]
            
        name=x['title']['text']
        profile_url=x['navigationUrl']
        #Headline=k['data']['elements'][0]['elements'][i]['snippetText']['text']
        Exp=x['headline']['text']
        location=x['subline']['text']
        if(name is not None):
            names.append(name)
        else:
            names.append('No Name')
        if(profile_url is not None):
            urls.append(profile_url)
        else:
            urls.append("No URL")
        if(Exp is not None):
            exp.append(Exp)
        else:
            exp.append('No Exp')
        if(location is not None):
            locs.append(location)
        else:
            locs.append('No Location')
    df=pd.DataFrame()
    df['name']=names
    df['Profile Url']=urls
    df['exp']=exp
    df['location']=locs
    return df

In [6]:
def id_generator(soup):
    for i in soup.findAll('code'):
        if(re.match(r'^\n  {"request":"/voyager/api/search/blended"*',i.text)):
            k=json.loads(i.text)['body']
            return k

In [9]:
soup

<!DOCTYPE html>
<html lang="en"><head>
<script type="application/javascript">!function(i,n){void 0!==i.addEventListener&&void 0!==i.hidden&&(n.liVisibilityChangeListener=function(){i.hidden&&(n.liHasWindowHidden=!0)},i.addEventListener("visibilitychange",n.liVisibilityChangeListener))}(document,window);</script>
<title>LinkedIn</title>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta class="mercado-icons-sprite" content="https://static-exp1.licdn.com/sc/h/c00rbckz4bb0q2l3vit7q6yfg" id="artdeco-icons/static/images" name="asset-url"/>
<meta content="" name="description"/>
<meta content="notranslate" name="google"/>
<meta content="false" name="is-http2"/>
<meta content="voyager-web" name="service"/>
<meta content="#0077B5" name="theme-color"/>
<meta content="width=device-width, initial-scale=1.0, minimum-scale=1.0" name="viewport"/>
<link href="/voyager/manifest.json" rel="manifest"/>
<link href="https://static-exp1.licdn.com/sc/h/2892cjpzj98fe8s47zqkgg

In [7]:
def main_page(keyword,pagenumber):
    dff=pd.DataFrame()
    link=conns(keyword)
    for p in range(1,pagenumber+1):
        soup=url_construction(link,p)
        cid=id_generator(soup)
        data=soup.find_all('code',attrs={'id':cid})[0]
        for i in data:
            k=json.loads(i)
        m=get_df(k,p)
        dff=pd.concat([dff,m],axis=0,ignore_index=True)
    return dff

In [9]:
keyword=input('Enter the Keyword: ')
pages=int(input('Enter the o of pages: '))
if keyword is not None and keyword !='':
    df=main_page(keyword,pages)
else:
    print('Sorry')

Enter the Keyword:  Reinforcement Learning
Enter the o of pages:  5


https://www.linkedin.com/search/results/people/?keywords=Reinforcement%20Learning&origin=SUGGESTION&page=1
https://www.linkedin.com/search/results/people/?keywords=Reinforcement%20Learning&origin=SUGGESTION&page=2
https://www.linkedin.com/search/results/people/?keywords=Reinforcement%20Learning&origin=SUGGESTION&page=3
https://www.linkedin.com/search/results/people/?keywords=Reinforcement%20Learning&origin=SUGGESTION&page=4
https://www.linkedin.com/search/results/people/?keywords=Reinforcement%20Learning&origin=SUGGESTION&page=5


In [11]:
#df=df.iloc[100:500]
df.head()
df.shape

(50, 4)

In [9]:
def leave_unknown_urls(urls):
    regex=re.compile('.*UNKNOWN$')
    urls=[i for i in urls if not regex.match(i)]
    return urls

In [10]:
urls=leave_unknown_urls(df['Profile Url'])

In [11]:
def get_json1(url):
    html = client.get(url).content
    soup = BeautifulSoup(html , "html.parser")
    for i in soup.findAll('code'):
        if(re.match(r'^\n  {"request":"/voyager/api/identity/dash/profiles.*.FullProfileWithEntities.*',i.text)):
            idd=json.loads(i.text)['body']
    
    data=soup.find_all(attrs={'id':idd})[0]
    for i in data:
        s=json.loads(i)
    return s['included']

In [12]:
def get_about(s1):
    for i in range(len(s1)):
        if(re.match('com.linkedin.voyager.dash.identity.profile.Profile$',s1[i]['$type'])):
            if 'summary' in s1[i].keys() and s1[i]['summary'] is not None:
                summary=s1[i]['summary']
                return summary
            else:
                summary='None'
                return summary

In [13]:
def get_educ(s1):
    ls=[]
    for i in range(len(s1)):
        d={}
        if(re.match('com.linkedin.voyager.dash.identity.profile.Education',s1[i]['$type'])):
            kk=s1[i]
            #print(kk.keys())
            if 'degreeName' in kk.keys() and 'schoolName' in kk.keys() and 'fieldOfStudy' in kk.keys() and 'dateRange' in kk.keys():
                if(kk['degreeName'] is not None):
                    d['DegreeName']=kk['degreeName']
                else:
                    d['DegreeName']='Not Mentioned'

                if(kk['schoolName'] is not None):
                    d['SchoolName']=kk['schoolName']
                else:
                    d['SchoolName']='Not Mentioned School'

                if(kk['fieldOfStudy'] is not None):
                    d['FieldOfStudy']=kk['fieldOfStudy']
                else:
                    d['FieldOfStudy']='Not Mentioned FS'
                    
                if kk['dateRange'] is not None:
                    if 'start' in kk['dateRange'].keys() and kk['dateRange']['start'] is not None:
                        if 'year' in kk['dateRange']['start'].keys() and kk['dateRange']['start']['year'] is not None:
                            d['Start Date']=kk['dateRange']['start']['year']
                            #dd.append(kk['dateRange']['start']['year'])
                        else:
                            d['Start Date']='No Start Date'
                    else:
                        d['Start Date']='No Start Key'
                        
                    if 'end' in kk['dateRange'].keys() and kk['dateRange']['end'] is not None:
                        if 'year' in kk['dateRange']['end'].keys() and kk['dateRange']['end']['year'] is not None:
                            d['End Date']=kk['dateRange']['end']['year']
                        else:
                            d['End Date']='No end year'
                    else:
                    #l=kk['dateRange']['end']['year']='No end year'
                        d['End Date']='No end key'


            #k=sn+"-"+dn+"-"+fs
            #ls.append(i['schoolName'])
            #ls.append(i['degreeName'])
            #ls.append(i['fieldOfStudy'])
            ls.append(d)
    return ls

In [14]:
def get_mm(dd):
    ff={}
    current_month=int(datetime.datetime.today().month)  
    sm=dd['Start Month']
    em=dd['End Month']
    #print(sm,em)

    if sm!='No DateRange Start Month Key' or em!='No DateRange End Month Key':
        if sm!='No Start Month':
            if em!='No End Month' and em!='No end month':
                if em!='Present':
                    if(em<sm):
                        ff['m']=em-sm+1
                    else:
                        ff['m']= np.abs(em-sm)+1
                else:
                    ff['m']=current_month-sm+1
            else:
                ff['m']=0
        else:
                ff['m']= 0
    else:
        ff['m']= 0
    return ff


In [15]:
def get_exp(s1):
    exps=[]
    current_year=int(datetime.datetime.today().year)
    current_month=int(datetime.datetime.today().month)
    for i in range(len(s1)):
        d={}
        if re.match('com.linkedin.voyager.dash.identity.profile.Position$',s1[i]['$type']):
            kk=s1[i]
            if 'title' in kk.keys() and 'companyName' in kk.keys() and kk['title'] is not None and 'dateRange' in kk.keys():
                if kk['companyName'] is not None:
                    d['Title']=kk['title']
                    d['Company Name']=kk['companyName']
                else:
                    d['Title']=kk['title']
                    d['Company Name']='No Company Name'
                
                if kk['dateRange'] is not None:
                    if 'start' in kk['dateRange'].keys() and kk['dateRange']['start'] is not None:
                        if 'year' in kk['dateRange']['start'].keys() and kk['dateRange']['start']['year'] is not None:
                            d['Start Year']=kk['dateRange']['start']['year']
                            #dd.append(kk['dateRange']['start']['year'])
                        else:
                            d['Start Year']='No Start Year'
                            
                        if 'month' in kk['dateRange']['start'].keys() and kk['dateRange']['start']['month'] is not None:
                            d['Start Month']=kk['dateRange']['start']['month']
                            #dd.append(kk['dateRange']['start']['year'])
                        else:
                            d['Start Month']='No Start Month'
                             
                    else:
                        d['Start Year']='No Start Year Key'
                        d['Start Month']='No Start Month Key'
                        
                    if 'end' in kk['dateRange'].keys() and kk['dateRange']['end'] is not None:
                        if 'year' in kk['dateRange']['end'].keys() and kk['dateRange']['end']['year'] is not None:
                            d['End Year']=kk['dateRange']['end']['year']
                        else:
                            d['End Year']='No end year'
                            
                        
                        if 'month' in kk['dateRange']['end'].keys() and kk['dateRange']['end']['month'] is not None:
                            d['End Month']=kk['dateRange']['end']['month']
                        else:
                            d['End Month']='No end month'
                        
                    else:
                    #l=kk['dateRange']['end']['year']='No end year'
                        d['End Year']='Present'
                        d['End Month']='Present'
                else:
                    d['Start Year']='No DateRange Value'
                    d['End Year']='No DateRange Value'
                
            else:
                d['Title']='No title key'
                d['Company Name']='No companyName key'
                d['Start Year']='No DateRange Start Key'
                d['End Year']='No DateRange End Key'
                d['Start Month']='No DateRange Start Month Key'
                d['End Month']='No DateRange End Month Key'
            #print(d)
            if d['Title']=='No title key':
                   continue
            
            #if d['Title']!='No Title Key':            
            if d['Start Year'] is not None and d['Start Year']!='No Start Year' and d['Start Year']!='No DateRange Value' and d['Start Year']!='No DateRange Start Key':
                if d['End Year']!='No end year':
                    d['Exp (in months)']=12*int(current_year-d['Start Year'])
                if d['End Year']!='Present':
                    d['Exp (in months)']=12*int(d['End Year']-d['Start Year'])
                else:
                    d['Exp (in months)']=12*int(current_year-d['Start Year'])
                #print(d)
                jjj=get_mm(d)
                d['Total Exp']=jjj['m']+d['Exp (in months)']
            
            if 'geoLocationName' in kk.keys() and kk['geoLocationName'] is not None:
                d['Location']=kk['geoLocationName']
            else:
                d['Location']='No Location'

            if 'description' in kk.keys() and kk['description'] is not None:
                d['Description']=kk['description']
            else:
                d['Description']='No Description'
            
            
            #print(tt,loc,dd,cc)
            #print('------------------------------------')
            #Job_details=tt+"-"+cc+"-"+loc+"-"+dd
            #print(Job_details)
            #if d['Title']
            exps.append(d)
    return exps

In [16]:
def get_skills(s1):
    skills=[]
    for i in range(len(s1)):
        if(re.match('com.linkedin.voyager.dash.identity.profile.Skill',s1[i]['$type'])):
            skills.append(s1[i]['name'])
    return skills

In [17]:
def get_certificates(s1):
    certificates=[]
    for i in range(len(s1)):
        ddd={}
        if(re.match('com.linkedin.voyager.dash.identity.profile.Certification',s1[i]['$type'])):
            kk=s1[i]
            if 'authority' in kk.keys() and 'name' in kk.keys() and kk['name'] is not None:
                if kk['authority'] is not None:
                    ddd['Name']=kk['name']
                    ddd['Authority']=kk['authority']
                    #kk=s1[i]['name']+"-"+s1[i]['authority']
                    certificates.append(ddd)
                else:
                    ddd['Name']=kk['name']
                    ddd['Authority']='No Authority'
                    #kk=s1[i]['name']+"-"+s1[i]['authority']
                    certificates.append(ddd)
                    #kk=s1[i]['name']+"-"+'No Auth'
                    #certificates.append(kk)
    return certificates

In [18]:
def get_volexp(s1):
    volexp=[]
    for i in range(len(s1)):
        if re.match('com.linkedin.voyager.dash.identity.profile.VolunteerExperience',s1[i]['$type']):
            d={}
            kk=s1[i]

            if 'role' in kk.keys() and 'companyName' in kk.keys():
                if kk['role'] is not None:
                    d['Role']=kk['role']
                else:
                    d['Role']='No Role'

                if kk['companyName'] is not None:
                    d['Company Name']=kk['companyName']

                else:
                    d['Company Name']='No Company Name'


            if 'description' in kk and kk['description'] is not None:
                d['Description']=kk['description']
            else:
                d['Description']='No Description'


            #print(tt,loc,dd,cc)
            #print('------------------------------------')
            #Job_details=tt+"-"+cc+"-"+loc+"-"+dd
            #print(Job_details)
            volexp.append(d)
    return volexp

In [19]:
def get_projects(s1):
    projects=[]
    for i in range(len(s1)):
        d={}
        if(re.match('com.linkedin.voyager.dash.identity.profile.Project',s1[i]['$type'])):
            if 'title' in s1[i] and 'description' in s1[i] and s1[i]['title'] is not None:
                if (s1[i]['description'] is not None):
                    d['Title']=s1[i]['title']
                    d['Description']=s1[i]['description']
                    #vv=s1[i]['title']+'-'+s1[i]['description']
                    projects.append(d)
                else:
                    d['Title']=s1[i]['title']
                    d['Description']='No Description'
                    #vv=s1[i]['title']+'-'+s1[i]['description']
                    projects.append(d)
                    #vv=s1[i]['title']+'-'+'No Descc'
                    #projects.append(vv)
    return projects

#projects          #com.linkedin.voyager.dash.identity.profile.Honor

In [20]:
def get_patents(s1):
    patents=[]
    for i in range(len(s1)):
        d={}
        if(re.match('com.linkedin.voyager.dash.identity.profile.Patent',s1[i]['$type'])):
            if 'title' in s1[i].keys() and 'description' in s1[i].keys() and s1[i]['title'] is not None:
                if (s1[i]['description'] is not None):
                    d['Title']=s1[i]['title']
                    d['Description']=s1[i]['title']
                    #vv=s1[i]['title']+'-'+s1[i]['description']
                    patents.append(d)
                else:
                    d['Title']=s1[i]['title']
                    d['Description']='No Description'
                    #vv=s1[i]['title']+'-'+s1[i]['description']
                    patents.append(d)
                    #vv=s1[i]['title']+'-'+'No Descc'
                    #patents.append(vv)
    return patents
            #com.linkedin.voyager.dash.identity.profile.Honor

In [21]:
def get_honors(s1):
    honors=[]
    for i in range(len(s1)):
        d={}
        if(re.match('com.linkedin.voyager.dash.identity.profile.Honor',s1[i]['$type'])):
            kk=s1[i]
            if 'title' in kk.keys() and 'description' in kk.keys() and kk['title'] is not None:
                if (kk['description'] is not None):
                    d['Title']=kk['title']
                    d['Description']=kk['description']
                    #vv=s1[i]['title']+'-'+s1[i]['description']
                    honors.append(d)
                else:
                    d['Title']=kk['title']
                    d['Description']='No Description'
                    #vv=s1[i]['title']+'-'+s1[i]['description']
                    honors.append(d)
                    #vv=s1[i]['title']+'-'+'No Descc'
                    #honors.append(vv)
    return honors
 #com.linkedin.voyager.dash.identity.profile.Honor

In [22]:
def get_publication(s1):
    publications=[]
    for i in range(len(s1)):
        d={}
        if(re.match('com.linkedin.voyager.dash.identity.profile.Publication',s1[i]['$type'])):
            if 'name' in s1[i].keys() and 'publisher' in s1[i].keys() and s1[i]['name'] is not None:
                if (s1[i]['publisher'] is not None):
                    d['Name']=s1[i]['name']
                    d['Publisher']=s1[i]['publisher']
                    #vv=s1[i]['name']+'-'+s1[i]['publisher']
                    publications.append(d)
                else:
                    d['Name']=s1[i]['name']
                    d['Publisher']='No Publisher Name'
                    #vv=s1[i]['name']+'-'+s1[i]['publisher']
                    publications.append(d)
                    #vv=s1[i]['name']+'-'+'No Descc'
                    #publications.append(vv)
    return publications

In [23]:
def open_to_work(s):
    for i in range(len(s)):
        if 'profilePicture' in s[i].keys() and s[i]['profilePicture'] is not None:
            pp=s[i]['profilePicture']
            if 'frameType' in pp.keys() and pp['frameType'] is not None:
                ft=s[i]['profilePicture']['frameType']
            else:
                ft='dd'
            return ft

In [24]:
def brain(urls):
    main_df=pd.DataFrame()
    for url in urls:
        print(url)
        df=pd.DataFrame()
        s=get_json1(url)
        #op=open_to_work(s)
    
       # if op is not None and op!='dd':
        df['Profile Url']=[url]
        df['About']=[get_about(s)]
        df['Education']=[get_educ(s)]
        df['Experience']=[get_exp(s)]
        df['Skills']=[get_skills(s)]
        df['Certificates']=[get_certificates(s)]
        df['Vol Exp']=[get_volexp(s)]
        df['Projects']=[get_projects(s)]
        df['Patents']=[get_patents(s)]
        df['Honors']=[get_honors(s)]
        df['Publications']=[get_publication(s)]
            #df=df.from_dict(d)
        main_df=pd.concat([main_df,df],axis=0,ignore_index=True)
    return main_df

In [25]:
pkk=brain(urls)

https://www.linkedin.com/in/sai-sekhar-667b26123
https://www.linkedin.com/in/jeeva-thangaraj-68690b180
https://www.linkedin.com/in/hema-varshini-perla-a1140a181
https://www.linkedin.com/in/swamy-reddamaina-316aa3124
https://www.linkedin.com/in/kaira-nithin-312a1a197
https://www.linkedin.com/in/krishnakanthmp
https://www.linkedin.com/in/virija-nandyala-89b854194
https://www.linkedin.com/in/guruteja
https://www.linkedin.com/in/vandana-pal-b42769187
https://www.linkedin.com/in/udaygirish-maradana
https://www.linkedin.com/in/jeevan-dath-allumallu-b77536100
https://www.linkedin.com/in/pratiknath
https://www.linkedin.com/in/biswajitsarangi
https://www.linkedin.com/in/sumanth18131a0363
https://www.linkedin.com/in/neha-duggirala
https://www.linkedin.com/in/gauravlochab487
https://www.linkedin.com/in/nitika-kamboj
https://www.linkedin.com/in/ravivarmathotakura
https://www.linkedin.com/in/abhishek-vardhan-8b71b0192
https://www.linkedin.com/in/rohan-raju
https://www.linkedin.com/in/janhawi08
http

In [26]:
main_df=pd.merge(df,pkk,on='Profile Url')

In [27]:
main_df

,name,Profile Url,exp,location,About,Education,Experience,Skills,Certificates,Vol Exp,Projects,Patents,Honors,Publications
0,Sai Sekhar,https://www.linkedin.com/in/sai-sekhar-667b26123,Machine Learning Associate at Amazon,"Vishakhapatnam Area, India",Machine Learning Engineer with a demonstrated ...,[{'DegreeName': 'Bachelor of Technology - BTec...,"[{'Title': 'Machine Learning Associate', 'Comp...","[Public Speaking, Microsoft PowerPoint, Matlab...",[],[],[],[],[],[]
1,Jeeva Thangaraj,https://www.linkedin.com/in/jeeva-thangaraj-68...,Machine learning Engineer,"Chennai Area, India",I have around 1 year of experience in Machine ...,[{'DegreeName': 'Bachelor of Engineering - BE'...,"[{'Title': 'Associate Software Engineer', 'Com...","[Github, Kubernetes, Python (Programming Langu...",[],[],[],[],[],[]
2,Hema Varshini Perla,https://www.linkedin.com/in/hema-varshini-perl...,Machine Learning Intern at Technocolabs | Pyth...,"Vishakhapatnam Area, India","Excellent capability to explore, learn and und...",[{'DegreeName': 'Bachelor of Technology - BTec...,"[{'Title': 'Machine Learning Developer', 'Comp...","[Project Management, Data Visualization, Micro...","[{'Name': 'Using Databases with Python', 'Auth...","[{'Role': 'Graphic Designer', 'Company Name': ...",[],[],[],[]
3,Swamy Reddamaina,https://www.linkedin.com/in/swamy-reddamaina-3...,Immediate Joining| Machine Learning Engineer| ...,"Hyderabad, Telangana, India",Machine Learning Engineer with 3 years of wor...,"[{'DegreeName': 'B.tech', 'SchoolName': 'CVSR ...","[{'Title': 'Machine Learning Engineer', 'Compa...","[Python (Programming Language), Deep Learning,...","[{'Name': 'Python Programming', 'Authority': '...",[],[],[],[],[]
4,Kaira Nithin,https://www.linkedin.com/in/kaira-nithin-312a1...,Artificial Intelligence || Machine Learning,"Hyderabad Area, India",None,"[{'DegreeName': 'MPC', 'SchoolName': 'Narayana...","[{'Title': 'Artificial Intelligence Intern', '...","[Research, Programming, C (Programming Languag...",[],[],[],[],[],[{'Name': 'User Preference multi-criteria reco...
5,Mp Krishna Kanth,https://www.linkedin.com/in/krishnakanthmp,Machine Learning Engineer | Data Science,"Visakhapatnam, Andhra Pradesh, India",None,"[{'DegreeName': 'Intermediate', 'SchoolName': ...","[{'Title': 'Core Team Member', 'Company Name':...","[CPP, Business Strategy, Database Management S...","[{'Name': 'Using Databases with Python', 'Auth...","[{'Role': 'Sponsorship Coordinator', 'Company ...",[{'Title': 'Health Insurance Cost Prediction '...,[],[],[]
6,Virija Nandyala,https://www.linkedin.com/in/virija-nandyala-89...,Machine Learning Aspirant,"Vishakhapatnam Area, India","""You can't finish what you don't start and you...",[{'DegreeName': 'Bachelor of Technology - BTec...,[],"[Machine Learning, C (Programming Language), P...","[{'Name': 'Python for Data science', 'Authorit...",[],[],[],[],[]
7,Ganesh Guru Teja Pulavarty,https://www.linkedin.com/in/guruteja,Machine Learning Developer,"Vishakhapatnam Area, India",Machine Learning and Cyber Security Enthusiast,"[{'DegreeName': 'Not Mentioned', 'SchoolName':...","[{'Title': 'Machine Learning Developer', 'Comp...","[Java, Microsoft Excel, Windows, Ethical Hacki...","[{'Name': 'EMC Academic Associate, Data Scienc...","[{'Role': 'Activist', 'Company Name': 'Free So...",[],[],[],[]
8,Vandana Pal,https://www.linkedin.com/in/vandana-pal-b42769187,Machine Learning Engineer | HackerRank - 5 sta...,"New Delhi, Delhi, India",I'm an Innovative and deadline-driven Machine ...,[{'DegreeName': 'Bachelor of Technology - BTec...,"[{'Title': 'Software Engineer Intern', 'Compan...","[SQL, C++, Machine Learning, TensorFlow, Data ...","[{'Name': 'Machine Learning with Python', 'Aut...","[{'Role': 'Placement Coordinator', 'Company Na...",[{'Title': 'Disaster Tweet analysis | Kaggle C...,[],"[{'Title': 'Problem Solving', 'Description': '...",[]
9,Uday Girish Maradana,https://www.linkedin.com/in/udaygirish-maradana,Machine Learning Engineer at Quantiphi,"Vishakhapatnam

In [34]:
def get_total_exp():
    ls=[]
    for i in main_df['Experience']:
        l=len(i)
        sums=0
        for j in range(l):
            sums+=i[j]['Total Exp']
        ls.append(sums)
    return ls

In [35]:
def get_total_desc():
    pk=[]
    for i in main_df['Experience']:
        
        l=len(i)
        ls=[]
        for j in range(l):
            ls.append(i[j]['Description'])
        pk.append(ls)
    return pk

In [36]:
Exp_df=pd.DataFrame()
Exp_df['Profile Url']=main_df['Profile Url']
Exp_df['About']=main_df['About']
Exp_df['Total_Exp']=get_total_exp()
Exp_df['Total Desc']=get_total_desc()

In [37]:
for i in range(len(Exp_df['Total Desc'])):
    Exp_df['Total Desc'][i].append(Exp_df['About'][i])

In [38]:
Exp_df.drop(columns=['About'],inplace=True)

In [39]:
sk=[]
for i in Exp_df['Total Desc']:
    sk.append([j for j in i if j!='None' and j!='No Description'])

In [40]:
Exp_df['Total Desc']=sk

In [41]:
pawan=[]
for i in range(len(Exp_df['Total Desc'])):
    gg=Exp_df['Total Desc'][i]
    #print(gg)
    z=' '.join(gg)
    z=z.strip()
    z=re.sub('\n','',z)
    pawan.append(z)

In [42]:
Exp_df['Total Desc']=pawan

In [ ]:
#main_df.to_csv('Main DataFrame.csv')

In [ ]:
#Exp_df.to_csv('Exp DataFrame.csv')

In [ ]:
main_df.shape

#### You can remove the comments of the above to cells so that the Complete data and the Experience related data are stored in two different csv files.